### <CENTER>ML ZOOMCAMP 2025 </CENTER>
### <CENTER>06 Trees - Homework</CENTER>
### <CENTER>ANGOLE DANIEL</CENTER>

#### Dataset

In this homework, we continue using the fuel efficiency dataset. Download it from here.

You can do it with wget:

wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
The goal of this homework is to create a regression model for predicting the car fuel efficiency (column 'fuel_efficiency_mpg').


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

In [2]:
!wget 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'

--2025-11-08 20:45:03--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘car_fuel_efficiency.csv.1’

car_fuel_efficiency 100%[===================>] 853.70K  2.89MB/s    in 0.3s    

2025-11-08 20:45:04 (2.89 MB/s) - ‘car_fuel_efficiency.csv.1’ saved [874188/874188]



In [3]:
df = pd.read_csv("car_fuel_efficiency.csv")

In [4]:
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


#### Preparing the dataset

<b> Preparation: </b>

- Fill missing values with zeros.
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the train_test_split function and set the random_state parameter to 1.
- Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [5]:
df.isna().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [6]:
df = df.fillna(0)

In [7]:
df.isna().sum()

engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

In [8]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train['fuel_efficiency_mpg'].values
y_val = df_val['fuel_efficiency_mpg'].values
y_test = df_test['fuel_efficiency_mpg'].values

del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

dv = DictVectorizer(sparse=True)

train_dicts = df_train.to_dict(orient='records')
val_dicts = df_val.to_dict(orient='records')
test_dicts = df_test.to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

<b>Question 1</b>

Let's train a decision tree regressor to predict the fuel_efficiency_mpg variable.
- Train a model with max_depth=1.

    Which feature is used for splitting the data?

        - 'vehicle_weight'
        - 'model_year'
        - 'origin'
        - 'fuel_type'

In [9]:
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

,criterion,'squared_error'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,1
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [10]:
print(export_text(dt, feature_names=list(dv.get_feature_names_out())))

|--- vehicle_weight <= 3022.11
|   |--- value: [16.88]
|--- vehicle_weight >  3022.11
|   |--- value: [12.94]



<b>Question 2</b>

Train a random forest regressor with these parameters:

- n_estimators=10
- random_state=1
- n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on the validation data?
- 0.045
- 0.45
- 4.5
- 45.0

In [11]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
y_val_pred = rf.predict(X_val)

rmse = round(root_mean_squared_error(y_val, y_val_pred), 3)
print(f'RMSE: {rmse}')

RMSE: 0.46


<b> Question 3 </b>

Now let's experiment with the n_estimators parameter
- Try different values of this parameter from 10 to 200 with step 10.
- Set random_state to 1.
- Evaluate the model on the validation dataset.

After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for calculating the answer.
- 10
- 25
- 80
- 200

If it doesn't stop improving, use the latest iteration number in your answer.

In [12]:
array_n_estimators = np.arange(10, 201, 10)
for n in array_n_estimators:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)

    y_val_pred = rf.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_val_pred)
    print(f'n {n}, RMSE: {rmse:.3f}')

n 10, RMSE: 0.460
n 20, RMSE: 0.454
n 30, RMSE: 0.452
n 40, RMSE: 0.449
n 50, RMSE: 0.447
n 60, RMSE: 0.445
n 70, RMSE: 0.445
n 80, RMSE: 0.445
n 90, RMSE: 0.445
n 100, RMSE: 0.445
n 110, RMSE: 0.444
n 120, RMSE: 0.444
n 130, RMSE: 0.444
n 140, RMSE: 0.443
n 150, RMSE: 0.443
n 160, RMSE: 0.443
n 170, RMSE: 0.443
n 180, RMSE: 0.442
n 190, RMSE: 0.442
n 200, RMSE: 0.442


<b>Question 4</b>

Let's select the best max_depth:
- Try different values of max_depth: [10, 15, 20, 25]
- For each of these values,
    - try different values of n_estimators from 10 till 200 (with step 10)
    - calculate the mean RMSE
- Fix the random seed: random_state=1
What's the best max_depth, using the mean RMSE?
- 10
- 15
- 20
- 25

In [13]:
array_max_depth = [10, 15, 20, 25]

for depth in array_max_depth:
    arr_rmse = []
    for n in array_n_estimators:
        rf = RandomForestRegressor(n_estimators=n, max_depth=depth, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)

        y_val_pred = rf.predict(X_val)

        rmse = root_mean_squared_error(y_val, y_val_pred)
        arr_rmse.append(rmse)

    print(f'max_depth {depth}, Avg RMSE: {np.mean(arr_rmse):.3f}')

max_depth 10, Avg RMSE: 0.442
max_depth 15, Avg RMSE: 0.445
max_depth 20, Avg RMSE: 0.446
max_depth 25, Avg RMSE: 0.446


<b>Question 5<b>

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

- Train the model with these parameters:
    - n_estimators=10,
    - max_depth=20,
    - random_state=1,
    - n_jobs=-1 (optional)
- Get the feature importance information from this model

What's the most important feature (among these 4)?
- vehicle_weight
- horsepower
- acceleration
- engine_displacement

In [14]:
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

feat_importances = rf.feature_importances_

feature_names = dv.get_feature_names_out()
max_val = feat_importances.max()
max_idxs = np.flatnonzero(feat_importances == max_val)
for i in max_idxs:
    print(feature_names[i], ": ", feat_importances[i])

vehicle_weight :  0.9591499647407432


<b> Question 6</b>

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

- Install XGBoost
- Create DMatrix for train and validation
- Create a watchlist
- Train a model with these parameters for 100 rounds:

            xgb_params = {
                'eta': 0.3, 
                'max_depth': 6,
                'min_child_weight': 1,
                
                'objective': 'reg:squarederror',
                'nthread': 8,
                
                'seed': 1,
                'verbosity': 1,
            }

Now change eta from 0.3 to 0.1.

Which eta leads to the best RMSE score on the validation dataset?

- 0.3
- 0.1
- Both give equal value

In [15]:
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=dv.feature_names_)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=dv.feature_names_)

watchlist = [(dtrain, 'train'), (dval, 'eval')]

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

evals_result = {}

model = xgb.train(xgb_params, dtrain, num_boost_round=100, 
                  evals_result=evals_result,
                  evals=watchlist)

[0]	train-rmse:1.81393	eval-rmse:1.85444
[1]	train-rmse:1.31919	eval-rmse:1.35353
[2]	train-rmse:0.98120	eval-rmse:1.01316
[3]	train-rmse:0.75443	eval-rmse:0.78667
[4]	train-rmse:0.60680	eval-rmse:0.64318
[5]	train-rmse:0.51381	eval-rmse:0.55664
[6]	train-rmse:0.45470	eval-rmse:0.50321
[7]	train-rmse:0.41881	eval-rmse:0.47254
[8]	train-rmse:0.39534	eval-rmse:0.45509
[9]	train-rmse:0.38038	eval-rmse:0.44564
[10]	train-rmse:0.37115	eval-rmse:0.43896
[11]	train-rmse:0.36361	eval-rmse:0.43594
[12]	train-rmse:0.35850	eval-rmse:0.43558
[13]	train-rmse:0.35365	eval-rmse:0.43394
[14]	train-rmse:0.35025	eval-rmse:0.43349
[15]	train-rmse:0.34666	eval-rmse:0.43362
[16]	train-rmse:0.34459	eval-rmse:0.43378
[17]	train-rmse:0.34128	eval-rmse:0.43405
[18]	train-rmse:0.33822	eval-rmse:0.43391
[19]	train-rmse:0.33709	eval-rmse:0.43374
[20]	train-rmse:0.33553	eval-rmse:0.43376
[21]	train-rmse:0.33243	eval-rmse:0.43453
[22]	train-rmse:0.33031	eval-rmse:0.43510
[23]	train-rmse:0.32815	eval-rmse:0.43601
[2

In [16]:
y_val_pred = model.predict(dval)

rmse = root_mean_squared_error(y_val, y_val_pred)
print(f'RMSE: {round(rmse, 3)}')

RMSE: 0.45


In [17]:
xgb_params2 = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

evals_result2 = {}

model2 = xgb.train(xgb_params2, dtrain, num_boost_round=100, 
                  evals_result=evals_result2,
                  evals=watchlist)

[0]	train-rmse:2.28944	eval-rmse:2.34561
[1]	train-rmse:2.07396	eval-rmse:2.12434
[2]	train-rmse:1.88066	eval-rmse:1.92597
[3]	train-rmse:1.70730	eval-rmse:1.74987
[4]	train-rmse:1.55163	eval-rmse:1.59059
[5]	train-rmse:1.41247	eval-rmse:1.44988
[6]	train-rmse:1.28796	eval-rmse:1.32329
[7]	train-rmse:1.17660	eval-rmse:1.20930
[8]	train-rmse:1.07736	eval-rmse:1.10830
[9]	train-rmse:0.98883	eval-rmse:1.02009
[10]	train-rmse:0.91008	eval-rmse:0.94062
[11]	train-rmse:0.84030	eval-rmse:0.87100
[12]	train-rmse:0.77874	eval-rmse:0.80916
[13]	train-rmse:0.72417	eval-rmse:0.75465
[14]	train-rmse:0.67626	eval-rmse:0.70780
[15]	train-rmse:0.63402	eval-rmse:0.66672
[16]	train-rmse:0.59690	eval-rmse:0.63062
[17]	train-rmse:0.56447	eval-rmse:0.60016
[18]	train-rmse:0.53619	eval-rmse:0.57383
[19]	train-rmse:0.51138	eval-rmse:0.55044
[20]	train-rmse:0.48983	eval-rmse:0.53064
[21]	train-rmse:0.47135	eval-rmse:0.51451
[22]	train-rmse:0.45501	eval-rmse:0.49998
[23]	train-rmse:0.44120	eval-rmse:0.48790
[2

In [18]:
y_val_pred = model.predict(dval)

rmse = root_mean_squared_error(y_val, y_val_pred)
print(f'RMSE: {round(rmse, 3)}')

RMSE: 0.45
